In [1]:
import os
import cv2
import base64
import requests
import pickle

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

In [11]:
path = "DataSet/train" # กำหนด path เพื่อเข้าไปดึงข้อมูลรูปภาพ
list_x = []
list_y = []

for sub in os.listdir(path): # ดึงรูปภาพมาเก็บไว้ใน list_x และชื่อโฟลเดอร์มาเก็บใน list_y
    for fn in os.listdir(os.path.join(path, sub)):
        path_file_img = os.path.join(path, sub, fn)
        readImage = cv2.imread(path_file_img, cv2.IMREAD_GRAYSCALE)
        list_x.append(readImage)
        list_y.append(sub)

In [12]:
def imgtovec(img): # สร้าง function ขึ้นมาเพื่อเรียกใช้ api ที่สร้างไว้
    try:
        resized_img = cv2.resize(img, (128, 128), cv2.INTER_AREA)
        v, buffer = cv2.imencode(".jpg", resized_img)
        img_str = base64.b64encode(buffer).decode('utf-8')
        image_data_string = img_str

        url = "http://localhost:80/api/genhog"
        params = {"img": image_data_string}

        response = requests.get(url, json=params)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"เรียก API ไม่สำเร็จ : {response.status_code}"}
    except Exception as ex:
        return {"error": f"เกิดข้อผิดพลาด: {str(ex)}"}

# เรียกใช้ api จะได้ข้อมูลภาพที่เป็น hog มาและเก็บใน hogvector
hogvectors = []
for i in range(len(list_x)):
    res = imgtovec(list_x[i])
    vec = list(res["hog"])
    vec.append(list_y[i])
    hogvectors.append(vec)



In [13]:
# เขียน hogvectors_train.pkl ขึ้นมา
write_path = "hogvectors_train.pkl"
pickle.dump(hogvectors, open(write_path, "wb"))
print("data preparation is done")

data preparation is done


In [33]:
hogvectors_train = pickle.load(open('hogvectors_train.pkl', 'rb'))
hogvectors_test = pickle.load(open('hogvectors_test.pkl', 'rb'))

# ดึงข้อมูลทุกแถว และ คอลัมน์ที่ 0-8099 มาเป็น feature 
X_train_data = [hogfeature_Xtrain[0:8100] for hogfeature_Xtrain in hogvectors_train]
X_test_data = [hogfeature_Xtest[0:8100] for hogfeature_Xtest in hogvectors_test]

# ดึงข้อมูลทุกแถว แต่เอาแค่คอลัมน์สุดท้าย มาเป็น class
Y_train_data = [hogfeature_Ytrain[-1] for hogfeature_Ytrain in hogvectors_train]
Y_test_data = [hogfeature_Ytest[-1] for hogfeature_Ytest in hogvectors_test]

label_encoder = LabelEncoder() # สร้าง object label_encoder จาก Class LabelEncoder เพราะต้องใช้ในการแปลงชื่อยี่ห้อรถยนต์เป็นตัวเลข
y_cls_train = label_encoder.fit(Y_train_data) # ใช้ .fit(Y_train_data) เพื่อใช้ในการเรียนรู้ว่า ชื่อยี่ห้อรถยนต์จะถูกแทนด้วยตัวเลขอะไรบ้าง ( ทำ mapping )
y_labelNum_train = label_encoder.transform(Y_train_data) # หลังจากใช้ .fit เพื่ออบรมแล้วจะเป็นตัวเลขตาม mapping ที่ถูกสร้างไว้
y_cls_test = label_encoder.fit(Y_test_data)
y_labelNum_test = label_encoder.transform(Y_test_data)

In [34]:
# สร้าง object จาก model DecisionTree
clf = DecisionTreeClassifier(random_state=42)

# สร้าง object จาก modelXGBoost
xgb_model = xgb.XGBClassifier(objective="multi:softmax",num_class=len(label_encoder.classes_), random_state=42)

# ทำการรวม 2โมเดล ที่สร้างไว้มารวมด้วยกัน 
ensemble_model = VotingClassifier(estimators=[('DecisionTree', clf), ('XGBoost', xgb_model)], voting='hard',weights=[1, 4])

# .fit() X_train_data เป็นการให้โมเดลมันเรียนรู้ข้อมูลที่เหมาะสมกับข้อมูล ส่วน Y_train_data เป็นคำตอบที่ควรจะได้จากการเรียนรู้
ensemble_model.fit(X_train_data, y_labelNum_train) 

# ใช้ข้อมูลทดสอบ X_test_data เพื่อไว้ทำนายผลลัพธ์ที่ได้จากโมเดลนี้ของข้อมูลทดสอบ
y_pred = ensemble_model.predict(X_test_data) 

In [26]:
accuracy = accuracy_score(y_labelNum_test, y_pred)
confusionMatrix = confusion_matrix(y_labelNum_test, y_pred)
print("Accuracy: ", accuracy)
print("Confusion Matrix: ", confusionMatrix)

Accuracy:  0.6555965559655597
Confusion Matrix:  [[163   1   4   5   2   6  18]
 [  8  39   0   2   3   0  15]
 [ 12   0  33   0   2   3  25]
 [ 41   0   2  15   3   3  10]
 [ 13   5   3   1  56   0  24]
 [ 16   1   2   1   3  74   9]
 [ 25   2   0   2   6   2 153]]


In [35]:
# เขียน model_genhog.pkl ขึ้นมา
path_model = 'model_genhog.pkl'
pickle.dump(ensemble_model, open(path_model, 'wb'))